In [1]:
import pandas as pd
import re
import urllib.request
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import os
import requests
import urllib
import html2text
from datetime import datetime
from pytz import timezone
from tzlocal import get_localzone
h = html2text.HTML2Text()
h.ignore_links=True
pd.low_memory=False

df = pd.read_csv("C:/Users/VickyViper/Documents/URL categorization application/url-classification-dataset-dmoz/URL Classification.csv")
df = df.drop('1',axis=1)
df.columns=['url','category']


In [ ]:
def extract(url):
    try:
        page = requests.get(url)
        if page.status_code == 200:
            contents = page.content
            soup = BeautifulSoup(contents)
            [script.decompose() for script in soup(["script", "style"])]
            strips = list(soup.stripped_strings)
            strips = ' '.join(strips)
            return strips
        else:
            return "NULL"
    except:
        return "NULL"
    
    
def extract_1(url):
    try:
        html = urllib.request.urlopen(url)
        soup = BeautifulSoup(html)
        data = soup.findAll(text=True)
        def visible(element):
            if element.parent.name in ['style', 'script', '[document]', 'head', 'title']:
                return False
            elif re.match('<!--.*-->', str(element.encode('utf-8'))):
                return False
            return True
        result = filter(visible, data)
        return ' '.join(result)
    except:
        return 'NULL'

def extract_link(url):
    try:
        page = urllib.request.urlopen(url)
    except:
        return "NULL"
    data=page.read()
    soup=BeautifulSoup(data,'html.parser')
    tags = ['title','a','p','input','div','html']
    result = []
    for k in tags:
        fi=soup.find_all(k)
        s = [h.handle(str(i)) for i in fi]
        s = ' '.join(s)
        s = s.replace('\n',' ')
        s = re.sub('[^A-Za-z0-9]+', ' ',s)
        result.append(s)
    return ' '.join(result)

def prepare_url(url):
    urls =[]
    secure_scheme = 'https://'
    insecure_scheme = 'http://'
    netloc =  urlparse(url).netloc
    if len(netloc) < 2:
            netloc = urlparse(url).path
    
    urls.append(secure_scheme+netloc)
    urls.append(insecure_scheme+netloc)
    return urls

def create_df(i):
    url = prepare_url(df.url[i])
    try:
        text =  extract_link(url[0])
        if text == 'NULL':
            text = extract_link(url[1])
        if text == 'NULL':
            text = extract(url[0])
        if text == 'NULL':
            text = extract(url[1])
        if text == 'NULL':
            text = extract_1(url[0])
        if text == 'NULL':
            text = extract_1(url[1])
        data.loc[i] = [df.url[i],df.category[i],text]
    except:
        data.loc[i] = [df.url[i],df.category[i],"NULL"]
    print((i/62519)*100," % complete",end='\r')


data = pd.DataFrame(columns=['url','category','text'])
print("You are given with 1.56 million websites and they are divided into 25 parts to extract data.")
section = int(df.shape[0]/25)
part = int(input("please type any number 0 - 24 to finish"))
begin = section * part
end =  begin + section
print("You part is: ",part," so the range is ",begin,"-",end)
format = "%Y-%m-%d %H:%M:%S"
now_utc = datetime.now(timezone('GMT'))
now_local = now_utc.astimezone(get_localzone())
print("Program started at",now_local.strftime(format))

for i in range(begin,end):
        while threading.active_count() > 100:
            time.sleep(1)
        t =  threading.Thread(target=create_df,args=(i,))
        t.start()

while data.shape[0] < section-10:
        time.sleep(1)
        
now_utc = datetime.now(timezone('GMT'))
now_local = now_utc.astimezone(get_localzone())
print("Program finished at",now_local.strftime(format))

print("saving data to CSV file")
data.to_csv(str(part)+"_data.csv")

You are given with 1.5 million websites and they are divided into 25 parts to extract data.
